## 1. 创建 FP 树的数据结构

In [5]:
class treeNode:
    def __init__(self, name, num, parentNode):
        self.name = name
        self.num = num
        self.parent = parentNode
        self.nodeLink = None
        self.children = {}
    def __repr__(self):
        return f'{self.name} with number of {self.num}'

## 2. 构建 FP 树

In [38]:
def createTree(dataSet, minSup):
    """
    param1 Dict[frozenset, 1:int] dataSet
    param2 int  minSup: 最小支持度
    """
    item_count = {}#统计各项出现次数
    #第一次遍历，得到频繁一项集
    # TODO
    for transaction in dataSet:
        for item in transaction:
            # item_count: Dict[key: item, occurence: int]
            item_count[item] = item_count.setdefault(item,0) + 1
    headerTable={}
    #剔除不满足最小支持度的项
    # TODO
    for item, sigma_item in item_count.items():
        if sigma_item >= minSup:
            headerTable[item] = sigma_item
    freqItemSet = set(headerTable.keys())#满足最小支持度的频繁项集
    if len(freqItemSet) == 0:
        return None, None
    for k in headerTable:
        # 每个不同父结点的同项结点构成链表， 链表头为Null
        # headerTable[k] = [headerTable[k], treeNode('Null', 1, None)] # element: [count, node]
        headerTable[k] = [headerTable[k], None] # element: [count, node]
    
    tree_header = treeNode('Null',1,None)
    #第二次遍历，建树
    # TODO
    # 为尽可能公用祖先结点，对每条transaction的item进行排序，按支持度降序排列
    for trans in dataSet:
        ordered_items = sorted(trans, key=lambda x: headerTable.get(x,[0])[0], reverse=True)
        updateTree(ordered_items, tree_header, headerTable)
        

    return tree_header, headerTable

def updateTree(items, node, headerTable):
    # 先判断items[0]是否在头节点列表（频繁1-项集）
    if items[0] in headerTable:
        if items[0] in node.children:
            # 判断items的第一个结点是否已作为子结点
            node.children[items[0]].num+=1
        else:
            # 创建新的分支
            node.children[items[0]] = treeNode(items[0],1,node)
            # 更新相应频繁项集的链表，往后添加
            # TODO
            updateHeader(headerTable, node.children[items[0]], items[0])
            # if items[0] in headerTable.keys():
            #     headerTable[items[0]][0] += 1

            #     if headerTable[items[0]][1] == None: 
            #         headerTable[items[0]][1] = node.children[items[0]]

            # updateHeader(headerTable[items[0]][1], node.children[items[0]])
        # 递归
        if len(items) > 1:
            updateTree(items[1:], node.children[items[0]], headerTable)

def updateHeader(headerTable, targetNode, item):#更新headertable中的node节点形成的链表
    # 初始为NoneType
    node = headerTable[item][1]
    if node == None:
        headerTable[item][1] = targetNode
    else:
        while node.nodeLink != None:
            node = node.nodeLink
        node.nodeLink = targetNode


In [40]:
myHeaderTab

{'r': [3, r with number of 1],
 'z': [5, z with number of 5],
 'y': [3, y with number of 3],
 'x': [4, x with number of 3],
 't': [3, t with number of 2],
 's': [3, s with number of 2]}

In [42]:
for trans in initSet:
	ordered_items = sorted(trans, key=lambda x: myHeaderTab.get(x,[0])[0], reverse=True)
	print(ordered_items)

['z', 'r', 'h', 'p', 'j']
['z', 'x', 'y', 't', 's', 'v', 'w', 'u']
['z']
['x', 'r', 's', 'o', 'n']
['z', 'x', 'y', 'r', 't', 'q', 'p']
['z', 'x', 'y', 't', 's', 'e', 'q', 'm']


In [14]:
def loadSimpDat():
    simpDat = [['r', 'z', 'h', 'j', 'p'],
               ['z', 'y', 'x', 'w', 'v', 'u', 't', 's'],
               ['z'],
               ['r', 'x', 'n', 'o', 's'],
               ['y', 'r', 'x', 'z', 'q', 't', 'p'],
               ['y', 'z', 'x', 'e', 'q', 's', 't', 'm']]
    return simpDat
 
def createInitSet(dataSet):
    retDict = {}
    for t in dataSet:
        retDict[frozenset(t)] = 1
    return retDict

In [15]:
simpDat = loadSimpDat()
initSet = createInitSet(simpDat)
initSet

{frozenset({'h', 'j', 'p', 'r', 'z'}): 1,
 frozenset({'s', 't', 'u', 'v', 'w', 'x', 'y', 'z'}): 1,
 frozenset({'z'}): 1,
 frozenset({'n', 'o', 'r', 's', 'x'}): 1,
 frozenset({'p', 'q', 'r', 't', 'x', 'y', 'z'}): 1,
 frozenset({'e', 'm', 'q', 's', 't', 'x', 'y', 'z'}): 1}

In [27]:
myFPtree, myHeaderTab = createTree(initSet, 3)

In [28]:
myHeaderTab

{'r': [3, r with number of 1],
 'z': [5, z with number of 5],
 'y': [3, y with number of 3],
 'x': [4, x with number of 3],
 't': [3, t with number of 2],
 's': [3, s with number of 2]}

In [29]:
node = myHeaderTab['r'][1]
while node.nodeLink != None:
	print(node)
	node = node.nodeLink

r with number of 1
r with number of 1


## 3. 抽取条件模式基
- 发现以给定元素项结尾的所有路径的函数

In [55]:
def find_cond_pattern_base(node_name, headerTable):
    Node = headerTable[node_name][1]
    cond_pat_base = {}
    # cond_pat_base = Dict[node.name: List[List[pattern]]]
    # TODO
    while Node != None:
        nodePath = []
        find_path(Node, nodePath)
        # 排除空路径，因为存在链表头节点
        if len(nodePath) > 0:
            cond_pat_base[frozenset(nodePath)] = Node.num
        Node = Node.nodeLink
    return cond_pat_base

def find_path(node, nodepath):
    # 迭代上溯整棵树
    if node.parent != None:
        nodepath.append(node.name)
        find_path(node.parent, nodepath)

In [56]:
find_cond_pattern_base('x', myHeaderTab)

{frozenset({'x', 'z'}): 3, frozenset({'x'}): 1}

In [57]:
find_cond_pattern_base('z', myHeaderTab)

{frozenset({'z'}): 5}

In [58]:
find_cond_pattern_base('r', myHeaderTab)

{frozenset({'r', 'z'}): 1,
 frozenset({'r', 'x'}): 1,
 frozenset({'r', 'x', 'y', 'z'}): 1}

In [59]:
find_cond_pattern_base('t', myHeaderTab)

{frozenset({'t', 'x', 'y', 'z'}): 2, frozenset({'r', 't', 'x', 'y', 'z'}): 1}

In [17]:
myHeaderTab

{'r': [3, Null with number of 1],
 'z': [5, Null with number of 1],
 'y': [3, Null with number of 1],
 't': [3, Null with number of 1],
 's': [3, Null with number of 1],
 'x': [4, Null with number of 1]}

## 4. 创建条件 FP 树

In [74]:
def create_cond_fptree(headerTable, min_support, temp, freq_items,support_data):
    """ 
    param3  Set[]   temp: 空集合，意义不明
    param5  Set[]   support_data: 空集合
    """
    # 最开始的频繁项集是headerTable中的各元素
    freqs = [v[0] for v in sorted(headerTable.items(), key=lambda p:p[1][0])] # 根据频繁项的总频次排序
    for freq in freqs: # 对每个频繁项
        freq_set = temp.copy()
        freq_set.add(freq)
        freq_items.add(frozenset(freq_set))
        #freq_items: Set[frozenset[freq 1-items]]
        if frozenset(freq_set) not in support_data:#检查该频繁项是否在support_data中
            support_data[frozenset(freq_set)]=headerTable[freq][0]
        else:
            support_data[frozenset(freq_set)]+=headerTable[freq][0]
        # support_data: Dict[freq 1-item, sigma]
        cond_pat_base = find_cond_pattern_base(freq, headerTable)#寻找到所有条件模式基
        cond_pat_dataset=[] #将条件模式基字典转化为数组
        for item in cond_pat_base:
            item_temp=list(item - frozenset([freq]))
            if item_temp == []: continue
            item_temp.sort()
            
            
            for i in range(cond_pat_base[item]):
                cond_pat_dataset.append(item_temp)
        #创建条件模式树
        # print(f'freq: {freq}')
        # print(f'cond_pat_base: {cond_pat_base}')
        # print(f'cond_pat_dataset: {cond_pat_dataset}')
        # print(f'min_support: {min_support}')
        cond_tree, cur_headtable = createTree(cond_pat_dataset, min_support)
        # print(f'cur_headtable: {cur_headtable}')
        # input()
        if cur_headtable != None:
            create_cond_fptree(cur_headtable, min_support, freq_set, freq_items,support_data) # 递归挖掘条件FP树


In [71]:
def generate_L(data_set,min_support):
    freqItemSet=set()
    support_data={}
    _, headerTable=createTree(data_set,min_support)
    #创建各频繁一项的fptree，并挖掘频繁项并保存支持度计数
    create_cond_fptree(headerTable, min_support, set(), freqItemSet,support_data)
    
    max_l=0
    for i in freqItemSet:#将频繁项根据大小保存到指定的容器L中
        if len(i)>max_l:max_l=len(i)
    L=[set() for _ in range(max_l)]
    for i in freqItemSet:
        L[len(i)-1].add(i)
    for i in range(len(L)):
        print("frequent item {}:{}".format(i+1,len(L[i]))) 
    return L,support_data 


In [75]:
generate_L(initSet, 3)

frequent item 1:6
frequent item 2:7
frequent item 3:4
frequent item 4:1


([{frozenset({'x'}),
   frozenset({'t'}),
   frozenset({'z'}),
   frozenset({'r'}),
   frozenset({'y'}),
   frozenset({'s'})},
  {frozenset({'x', 'z'}),
   frozenset({'y', 'z'}),
   frozenset({'t', 'x'}),
   frozenset({'t', 'z'}),
   frozenset({'s', 'x'}),
   frozenset({'t', 'y'}),
   frozenset({'x', 'y'})},
  {frozenset({'t', 'y', 'z'}),
   frozenset({'x', 'y', 'z'}),
   frozenset({'t', 'x', 'z'}),
   frozenset({'t', 'x', 'y'})},
  {frozenset({'t', 'x', 'y', 'z'})}],
 {frozenset({'r'}): 3,
  frozenset({'y'}): 3,
  frozenset({'x', 'y'}): 3,
  frozenset({'y', 'z'}): 3,
  frozenset({'x', 'y', 'z'}): 3,
  frozenset({'t'}): 3,
  frozenset({'t', 'x'}): 3,
  frozenset({'t', 'y'}): 3,
  frozenset({'t', 'x', 'y'}): 3,
  frozenset({'t', 'z'}): 3,
  frozenset({'t', 'x', 'z'}): 3,
  frozenset({'t', 'y', 'z'}): 3,
  frozenset({'t', 'x', 'y', 'z'}): 3,
  frozenset({'s'}): 3,
  frozenset({'s', 'x'}): 3,
  frozenset({'x'}): 4,
  frozenset({'x', 'z'}): 3,
  frozenset({'z'}): 5})